In [6]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

# =========================
# Helpers (stile come tuo)
# =========================

def k_from_h(h):
    h = pd.to_numeric(h, errors="coerce")
    if not np.isfinite(h) or h <= 0:
        return None
    if h < 1:
        return int(round(-math.log10(h)))
    if abs(h - round(h)) < 1e-12:
        return int(round(h))
    return int(round(h))

def sci3(x):
    try:
        x = float(x)
    except Exception:
        return "NaN"
    if not np.isfinite(x):
        return "NaN"
    if x == 0.0:
        return "$0$"
    exp = int(math.floor(math.log10(abs(x))))
    mant = x / (10**exp)
    mant = round(mant, 2)
    if abs(mant) >= 10:
        mant /= 10
        exp += 1
    return f"${mant:.2f} \\times 10^{{{exp}}}$"

def fmt_time(t):
    try:
        t = float(t)
    except Exception:
        return "NaN"
    if not np.isfinite(t):
        return "NaN"
    return f"{t:.4f}s" if t < 1 else f"{t:.2f}s"

def rate_mean(vals):
    vals = np.array(pd.to_numeric(vals, errors="coerce"), dtype=float)
    vals = vals[np.isfinite(vals)]
    vals = vals[(vals >= 0) & (vals <= 10)]
    return float(vals.mean()) if len(vals) else float("nan")

def mode_label(der_type, is_compwise):
    der_type = str(der_type).lower()
    is_cw = bool(int(is_compwise)) if not isinstance(is_compwise, bool) else is_compwise
    if der_type == "exact":
        return "exact"
    if der_type == "approx":
        return "full fd" + (" h prop" if is_cw else "")
    if der_type == "mixed":
        return "mixed fd" + (" h prop" if is_cw else "")
    return der_type

def mode_tag_for_filename(der_type, is_compwise):
    """Tag compatto, safe per filename."""
    der_type = str(der_type).lower()
    is_cw = int(is_compwise)
    if der_type == "exact":
        return "exact"
    if der_type == "approx":
        return "fullfd_hprop" if is_cw == 1 else "fullfd_const"
    if der_type == "mixed":
        return "mixedfd_hprop" if is_cw == 1 else "mixedfd_const"
    return f"{der_type}_cw{is_cw}"

def is_success_row(row):
    try:
        btflag = int(row["btflag"])
        iters  = int(row["iterations"])
        kmax   = int(row["kmax"])
        gn     = float(row["grad_norm"])
        tol    = float(row["tol"])
    except Exception:
        return False
    return (btflag == 1) and (iters < kmax) and (gn <= tol)

def success_mask(df_run):
    kmax = int(df_run["kmax"].iloc[0])
    tol  = float(df_run["tol"].iloc[0])
    return (df_run["btflag"] == 1) & (df_run["iterations"] < kmax) & (df_run["grad_norm"] <= tol)

def pick_base_row(df_run):
    # base: pert_id==1 se esiste
    if "pert_id" in df_run.columns:
        base = df_run[df_run["pert_id"] == 1]
        if len(base) > 0:
            return base.iloc[0]
        if df_run["pert_id"].notna().any():
            return df_run.loc[df_run["pert_id"].idxmin()]
    if "_rowidx" in df_run.columns:
        return df_run.sort_values("_rowidx").iloc[0]
    return df_run.iloc[0]

def build_block_verbose(df_run):
    """
    Ritorna:
      - metadata (prob, dim, mode, k_disp, kmax)
      - rows: base (\bar{x}) + pert 2..6 come 1..5
      - avg: media (success rows se esistono, altrimenti tutte) + succ_frac
    """
    base = pick_base_row(df_run)

    prob = int(base["problem_id"])
    dim  = int(base["n_dims"])
    der_type = str(base["derivative_type"]).lower()
    is_cw = int(base["is_compwise"])
    mode = mode_label(der_type, is_cw)

    kmax = int(base["kmax"])
    k_disp = None if der_type == "exact" else k_from_h(base.get("hG", np.nan))

    # ordina per pert_id (robusto) + ordine apparizione
    sort_cols = []
    if "pert_id" in df_run.columns:
        sort_cols.append("pert_id")
    if "_rowidx" in df_run.columns:
        sort_cols.append("_rowidx")
    df_run_sorted = df_run.sort_values(sort_cols, na_position="last").copy() if sort_cols else df_run.copy()

    sm = success_mask(df_run_sorted)
    n_total = int(df_run_sorted["pert_id"].nunique()) if "pert_id" in df_run_sorted.columns else int(len(df_run_sorted))
    n_succ  = int(sm.sum())
    succ_frac = f"{n_succ}/{n_total}"

    use = df_run_sorted[sm] if sm.any() else df_run_sorted

    avg = {
        "avg_grad": float(pd.to_numeric(use["grad_norm"], errors="coerce").mean()),
        "avg_it":   float(pd.to_numeric(use["iterations"], errors="coerce").mean()),
        "avg_rate": rate_mean(use.get("rate_of_conv", np.nan)),
        "avg_time": float(pd.to_numeric(use.get("execution_time", np.nan), errors="coerce").mean()),
        "kmax": kmax,
        "succ_frac": succ_frac
    }

    rows = []
    for _, r in df_run_sorted.iterrows():
        pid_raw = pd.to_numeric(r.get("pert_id", np.nan), errors="coerce")
        pid = int(pid_raw) if np.isfinite(pid_raw) else None

        if pid == 1:
            start_id = r"$\overline{x}$"
        elif pid is not None and pid >= 2:
            start_id = str(pid - 1)   # 2..6 -> 1..5
        elif pid is None:
            start_id = "NaN"
        else:
            start_id = str(pid)

        rate_val = pd.to_numeric(r.get("rate_of_conv", np.nan), errors="coerce")
        time_val = pd.to_numeric(r.get("execution_time", np.nan), errors="coerce")

        it_val = pd.to_numeric(r.get("iterations", np.nan), errors="coerce")
        it_val = int(it_val) if np.isfinite(it_val) else 0

        rows.append({
            "start_id": start_id,
            "grad": float(pd.to_numeric(r.get("grad_norm", np.nan), errors="coerce")),
            "it": it_val,
            "succ": "yes" if is_success_row(r) else "no",
            "rate": float(rate_val) if np.isfinite(rate_val) else float("nan"),
            "time": float(time_val) if np.isfinite(time_val) else float("nan"),
            "kmax": int(pd.to_numeric(r.get("kmax", kmax), errors="coerce")) if np.isfinite(pd.to_numeric(r.get("kmax", kmax), errors="coerce")) else kmax,
        })

    return {
        "prob": prob,
        "dim": dim,
        "mode": mode,
        "k": k_disp,
        "kmax": kmax,
        "rows": rows,
        "avg": avg
    }

def latex_table_verbose(blocks, include_k: bool):
    lines = []
    lines.append("\\begin{table}[H]")
    lines.append("\\centering")
    lines.append("\\renewcommand{\\arraystretch}{1.3}")
    lines.append("\\resizebox{\\textwidth}{!}{%")
    lines.append("\\arrayrulecolor{black!40}")

    if include_k:
        lines.append("\\begin{tabular}{|r||r||c||c||c||c||c||c||c||c||r|}")
        lines.append("\\hline")
        lines.append("\\rowcolor{gray!10}")
        lines.append("\\textbf{Prob.} & \\textbf{Dim.} & \\textbf{Mode} & \\textbf{start.pt ID} & \\textbf{k} & \\textbf{grad.norm} & \\textbf{iters/max} & \\textbf{succ.} & \\textbf{flag} & \\textbf{rate} & \\textbf{time} \\\\")
        lines.append("\\hline\\hline")

        for b in blocks:
            k_cell = f"{int(b['k'])}" if b["k"] is not None else "NaN"

            first = True
            for rr in b["rows"]:
                rate_cell = "NaN" if (not np.isfinite(rr["rate"])) else f"{rr['rate']:.2f}"

                pcell = str(b["prob"]) if first else ""
                dcell = str(b["dim"])  if first else ""
                mcell = str(b["mode"]) if first else ""
                first = False

                lines.append(
                    f"{pcell} & {dcell} & {mcell} & {rr['start_id']} & {k_cell} & "
                    f"{sci3(rr['grad'])} & {rr['it']}/{rr['kmax']} & {rr['succ']} &  & "
                    f"{rate_cell} & {fmt_time(rr['time'])} \\\\"
                )
                lines.append("\\hline")

            avg = b["avg"]
            rate_avg = "NaN" if (not np.isfinite(avg["avg_rate"])) else f"{avg['avg_rate']:.2f}"
            lines.append(
                f" &  &  & \\textit{{Avg (successes)}} & {k_cell} & "
                f"{sci3(avg['avg_grad'])} & {avg['avg_it']:.1f}/{avg['kmax']} & {avg['succ_frac']} &  & "
                f"{rate_avg} & {fmt_time(avg['avg_time'])} \\\\"
            )
            lines.append("\\thickhline")

    else:
        lines.append("\\begin{tabular}{|r||r||c||c||c||c||c||c||c||r|}")
        lines.append("\\hline")
        lines.append("\\rowcolor{gray!10}")
        lines.append("\\textbf{Prob.} & \\textbf{Dim.} & \\textbf{Mode} & \\textbf{start.pt ID} & \\textbf{grad.norm} & \\textbf{iters/max} & \\textbf{succ.} & \\textbf{flag} & \\textbf{rate} & \\textbf{time} \\\\")
        lines.append("\\hline\\hline")

        for b in blocks:
            first = True
            for rr in b["rows"]:
                rate_cell = "NaN" if (not np.isfinite(rr["rate"])) else f"{rr['rate']:.2f}"

                pcell = str(b["prob"]) if first else ""
                dcell = str(b["dim"])  if first else ""
                mcell = str(b["mode"]) if first else ""
                first = False

                lines.append(
                    f"{pcell} & {dcell} & {mcell} & {rr['start_id']} & "
                    f"{sci3(rr['grad'])} & {rr['it']}/{rr['kmax']} & {rr['succ']} &  & "
                    f"{rate_cell} & {fmt_time(rr['time'])} \\\\"
                )
                lines.append("\\hline")

            avg = b["avg"]
            rate_avg = "NaN" if (not np.isfinite(avg["avg_rate"])) else f"{avg['avg_rate']:.2f}"
            lines.append(
                f" &  &  & \\textit{{Avg (successes)}} & "
                f"{sci3(avg['avg_grad'])} & {avg['avg_it']:.1f}/{avg['kmax']} & {avg['succ_frac']} &  & "
                f"{rate_avg} & {fmt_time(avg['avg_time'])} \\\\"
            )
            lines.append("\\thickhline")

    lines.append("\\end{tabular}")
    lines.append("}")
    lines.append("\\end{table}")
    return "\n".join(lines)

# =========================
# MAIN: carica CSV e genera .tex singoli
# =========================

csv_path = Path("megacsv83_clean.csv")  # cambia se serve
df = pd.read_csv(csv_path)

# numerici (robusto)
for col in ["grad_norm","iterations","btflag","kmax","tol","hG","hH","is_compwise",
            "execution_time","rate_of_conv","pert_id","problem_id","n_dims","run_id"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# preserva ordine di apparizione
df = df.reset_index(drop=False).rename(columns={"index":"_rowidx"})

# OUTPUT folder + method tag (come screenshot: tables_mod / ..._mod.tex)
out_dir = Path("tables_mod")
out_dir.mkdir(parents=True, exist_ok=True)
method_tag = "mod"

# ---- 1) costruisci metadata per ogni run_id (una riga = un run)
run_meta = []
for rid, g in df.groupby("run_id", sort=False):
    base = pick_base_row(g)
    prob = int(base["problem_id"])
    dim  = int(base["n_dims"])
    der_type = str(base["derivative_type"]).lower()
    is_cw = int(base["is_compwise"])
    k_disp = None if der_type == "exact" else k_from_h(base.get("hG", np.nan))
    first_idx = int(g["_rowidx"].min())

    run_meta.append({
        "run_id": int(rid) if np.isfinite(rid) else rid,
        "prob": prob,
        "dim": dim,
        "der_type": der_type,
        "is_cw": is_cw,
        "k": k_disp,
        "first_idx": first_idx
    })

meta = pd.DataFrame(run_meta).sort_values("first_idx")

# ---- 2) raggruppa i run per "tabella finale" (prob, dim, mode, k)
# exact: k = None; non-exact: k distingue le tabelle
group_keys = ["prob","dim","der_type","is_cw","k"]
created = []

for key, gmeta in meta.groupby(group_keys, dropna=False, sort=False):
    prob, dim, der_type, is_cw, k_val = key

    # run_ids in ordine di apparizione
    run_ids = gmeta.sort_values("first_idx")["run_id"].tolist()

    blocks = []
    for rid in run_ids:
        df_run = df[df["run_id"] == rid].copy()
        blocks.append(build_block_verbose(df_run))

    include_k = (str(der_type).lower() != "exact")
    mode_tag  = mode_tag_for_filename(der_type, is_cw)

    # naming: Prob31_n2_exact_mod.tex  (come screenshot)
    # per FD: aggiungo _k8 per evitare collisioni tra diverse hG
    if include_k:
        if k_val is None or (isinstance(k_val, float) and not np.isfinite(k_val)):
            k_tag = "_kNaN"
        else:
            k_tag = f"_k{int(k_val)}"
    else:
        k_tag = ""

    fname = f"Prob{int(prob)}_n{int(dim)}_{mode_tag}{k_tag}_{method_tag}.tex"
    out_path = out_dir / fname

    table_tex = latex_table_verbose(blocks, include_k=include_k)
    out_path.write_text(table_tex + "\n", encoding="utf-8")

    created.append(str(out_path))

print(f"Generated {len(created)} table files in: {out_dir.resolve()}")
for p in created:
    print(" -", p)


Generated 52 table files in: C:\Users\Andrea\Desktop\TABLER\tables_mod
 - tables_mod\Prob83_n2_exact_mod.tex
 - tables_mod\Prob83_n1000_exact_mod.tex
 - tables_mod\Prob83_n10000_exact_mod.tex
 - tables_mod\Prob83_n100000_exact_mod.tex
 - tables_mod\Prob83_n2_fullfd_const_k4_mod.tex
 - tables_mod\Prob83_n2_fullfd_const_k8_mod.tex
 - tables_mod\Prob83_n2_fullfd_const_k12_mod.tex
 - tables_mod\Prob83_n1000_fullfd_const_k4_mod.tex
 - tables_mod\Prob83_n1000_fullfd_const_k8_mod.tex
 - tables_mod\Prob83_n1000_fullfd_const_k12_mod.tex
 - tables_mod\Prob83_n10000_fullfd_const_k4_mod.tex
 - tables_mod\Prob83_n10000_fullfd_const_k8_mod.tex
 - tables_mod\Prob83_n10000_fullfd_const_k12_mod.tex
 - tables_mod\Prob83_n100000_fullfd_const_k4_mod.tex
 - tables_mod\Prob83_n100000_fullfd_const_k8_mod.tex
 - tables_mod\Prob83_n100000_fullfd_const_k12_mod.tex
 - tables_mod\Prob83_n2_mixedfd_const_k4_mod.tex
 - tables_mod\Prob83_n2_mixedfd_const_k8_mod.tex
 - tables_mod\Prob83_n2_mixedfd_const_k12_mod.tex
 